# glm4 stream调用

In [1]:
import os
import torch
from threading import Thread
from transformers import AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, AutoModel

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
MODEL_PATH = '/opt/Data/ModelWeight/THUDM/glm-4-9b-chat'
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)
model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    device_map="cuda").eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []
max_length = 8192
top_p = 0.8
temperature = 0.6
stop = StopOnTokens()

user_input = "如何看待小孩子每天都喜欢玩，不太爱学习"
history.append([user_input, ""])
messages = []

for idx, (user_msg, model_msg) in enumerate(history):
    if idx == len(history) - 1 and not model_msg:
        messages.append({"role": "user", "content": user_msg})
        break
    if user_msg:
        messages.append({"role": "user", "content": user_msg})
    if model_msg:
        messages.append({"role": "assistant", "content": model_msg})

model_inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
).to(model.device)

streamer = TextIteratorStreamer(
    tokenizer=tokenizer,
    timeout=60,
    skip_prompt=True,
    skip_special_tokens=True
)

generate_kwargs = {
    "input_ids": model_inputs,
    "streamer": streamer,
    "max_new_tokens": max_length,
    "do_sample": True,
    "top_p": top_p,
    "temperature": temperature,
    "stopping_criteria": StoppingCriteriaList([stop]),
    "repetition_penalty": 1.2,
    "eos_token_id": model.config.eos_token_id,
}

t = Thread(target=model.generate, kwargs=generate_kwargs)
t.start()

print("GLM-4:", end="", flush=True)

for new_token in streamer:
    if new_token:
        print(new_token, end="", flush=True)
        history[-1][1] += new_token

history[-1][1] = history[-1][1].strip()

GLM-4:
孩子每天喜欢玩耍而不太热衷于学习的现象在儿童心理学中是正常且普遍的。以下是对这一现象的一些看法：

1. **生理和心理发展特点**：
   - 儿童时期是人类大脑发育最快的阶段之一，孩子们在这个时期的注意力、自控力和认知能力还在发展中。
   - 玩耍对于孩子的身心健康发展至关重要，它有助于他们探索世界、培养社交技能和解决问题的能力。

2. **教育理念**：
   - 当前的教育观念越来越强调“快乐学习”，即通过游戏化的方式让孩子在学习中获得乐趣，从而提高他们的学习兴趣和学习效果。
   - 强调素质教育而非单纯的应试教育，鼓励孩子在德智体美劳各方面全面发展。

3. **家庭教育与引导**：
   - 家长可以通过创造良好的家庭环境来激发孩子的学习兴趣，比如提供丰富的图书资源、科学实验器材等。
   - 通过亲子活动或参与社会实践等方式，将学习内容融入日常生活，使孩子在实际操作中发现知识的重要性。

4. **社会因素**：
   - 在竞争激烈的社会环境中，家长可能会对孩子有较高的期望值，这可能导致孩子感到压力重重，不愿意主动去学习。
   - 学校的教育方式和评价体系也可能影响学生的学习积极性。

5. **平衡玩耍与学习**：
   - 作为家长和教育者，应当找到玩耍与学习之间的平衡点，既不剥夺孩子应有的童年乐趣，也不忽视其学业成长。
   - 鼓励孩子进行有益身心的户外活动和集体运动，同时合理安排课业负担。

总之，对待孩子喜欢玩耍而较少热爱学习的情况，应该采取理解和支持的态度，结合科学的育儿方法，帮助孩子健康成长。在这个过程中，家长的耐心和理解尤为重要。

In [13]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

history = []
def runChat(user_input:str):
    history.append([user_input, ""])
    messages = []
    
    for idx, (user_msg, model_msg) in enumerate(history):
        if idx == len(history) - 1 and not model_msg:
            messages.append({"role": "user", "content": user_msg})
            break
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if model_msg:
            messages.append({"role": "assistant", "content": model_msg})
    
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        timeout=60,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 8192,
        "do_sample": True,
        "top_p": 0.8,
        "temperature":  0.6,
        "stopping_criteria": StoppingCriteriaList([StopOnTokens()]),
        "repetition_penalty": 1.2,
        "eos_token_id": model.config.eos_token_id,
    }
    
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    print("GLM-4:", end="", flush=True)

    generatetxt=""
    for new_token in streamer:
        if new_token:
            print(new_token, end="", flush=True)
            history[-1][1] += new_token
            generatetxt+=new_token
    t.join()
    return generatetxt

user_input = "如何看待少年谈朋友和有性行为"
res = runChat(user_input)
print(res)
history[-1][1] = history[-1][1].strip()

GLM-4:
关于青少年谈论恋爱以及涉及性行为的话题，这是一个复杂且敏感的社会问题。以下是从不同角度对此进行的一些看法：

1. **社会教育责任**：
   - 学校和家庭应承担起对青少年的性教育和心理健康教育的责任。
   - 应提供科学、客观的性知识普及，帮助青少年建立正确的价值观。

2. **法律与道德规范**：
   - 在中国，法律规定未成年人不得结婚，对于性行为也有相应的法律法规限制。
   - 道德上讲，尊重未成年人的身心发展规律是非常重要的。

3. **家庭教育**：
   - 家庭是孩子成长的第一课堂，家长应该以身作则，引导孩子树立正确的爱情观和人生观。
   - 家长应当关注孩子的情感需求，适时地与他们沟通，了解他们的想法和行为。

4. **心理支持**：
   - 对于涉世未深的青少年来说，面对感情和性的困惑时，他们需要得到适当的指导和心理支持。
   - 社会可以设立专门的心理咨询服务，帮助他们处理青春期的问题。

5. **媒体影响**：
   - 媒体在传播信息时应注意内容的适宜性，避免过度渲染或误导青少年。
   - 提倡健康向上的文化产品，减少不良信息的负面影响。

6. **社会责任**：
   - 全社会都应对青少年给予关爱和保护，共同营造一个有利于青少年健康成长的环境。

总之，对待这个问题要综合考虑多方面因素，既要保护未成年人的身心健康，又要正确引导他们对爱情和生活的理解。通过家庭、学校和社会的共同努力，可以帮助青少年形成健康的性观念，培养积极向上的人生态度。
关于青少年谈论恋爱以及涉及性行为的话题，这是一个复杂且敏感的社会问题。以下是从不同角度对此进行的一些看法：

1. **社会教育责任**：
   - 学校和家庭应承担起对青少年的性教育和心理健康教育的责任。
   - 应提供科学、客观的性知识普及，帮助青少年建立正确的价值观。

2. **法律与道德规范**：
   - 在中国，法律规定未成年人不得结婚，对于性行为也有相应的法律法规限制。
   - 道德上讲，尊重未成年人的身心发展规律是非常重要的。

3. **家庭教育**：
   - 家庭是孩子成长的第一课堂，家长应该以身作则，引导孩子树立正确的爱情观和人生观。
   - 家长应当关注孩子的情感需求，适时地与他们沟通，了解他们的想法和行为。

4. **心理支